In [334]:
import nltk.tokenize
import string
import itertools
import unicodedata
from collections import defaultdict, namedtuple
import re

## Task 1

In [301]:
Paragraph = namedtuple("Paragraph", ["entity", "content"])

with open("poczatki_wikipediowe.txt") as fstream:
    lines = list(map(lambda x: x.strip(), fstream.readlines()))
    paragraphs = []
    is_entity = True
    last_entity, last_content = "", ""
    for i in range(len(lines)):
        if is_entity:
            is_entity = False
            last_entity = lines[i][4:]
        elif lines[i] == "":
            is_entity = True
            paragraphs.append(Paragraph(entity=last_entity, content=last_content))
            last_entity, last_content = "", ""
        else:
            last_content += lines[i]
len(paragraphs)

1008132

In [386]:
m = re.search('([a-z]+)(\'*) lub (\'*)([a-z]+)', "''asd'' lub ''ced''")
m.group(4)

'ced'

In [462]:
for paragraph in paragraphs[:10000]:
    or_regex = re.search('(\'+)([^\']+)(\'+) lub (\'+)([^\']+)(\'+)', paragraph.content)
    zwana_regex = re.search(
        '(\'+)([^\']+)(\'+)([^\']+)zwana([^\']+)(\'*)([^\']+)(\'+)',
        paragraph.content
    )
    nazywana_regex = re.search(
        '(\'+)([^\']+)(\'+)([^\']+)nazywana([^\']+)(\'*)([^\']+)(\'+)',
        paragraph.content
    )
    entity_regex = re.search(
        'nazywana([^\']+)(\'+)([^\']+)(\'+)',
        paragraph.content
    )
    if zwana_regex:
        print(f"{zwana_regex.group(2)} # {zwana_regex.group(7)}")
    elif nazywana_regex:
        print(f"{nazywana_regex.group(2)} # {nazywana_regex.group(7)}")        
    elif entity_regex:
        print(f"{paragraph.entity} # {entity_regex.group(3)}")
    elif or_regex:
        print(f"{or_regex.group(2)} # {or_regex.group(5)}")

Alternatywa # suma logiczna
pismem żydowskim # pismem kwadratowym
Ajnowie # Ajnosi
Absinthe # Absinth
Adrenalina # epinefryną
Athene # Athenaie
behavior # behaviour
Canaletto # Canaletto młodszym
Pritani # Priteni
Britannia # Insula Albionum
Vaccinium vitis-idaea # Ericaceae
Control Program/Monitor # Control Program/Microcomputer
Tasunka Witko # Tashunka Uitko
Ndee # Dine
barwy procesowe # kolory triadowe
partial order # poset
Česko-Slovensko # Československo
kapitał rzeczowy # kapitał finansowy
Digital Video Disc # Digital Versatile Disc
deka # deko
ten, który coś rozdziela # ten, który coś przydziela
U # E sub w /sub 
I # χ
Emotikon # emotikona
Eurazja # Eurazja
teoria poznania # gnoseologia
predykat # forma zdaniowa
Fortran # FORTRAN
nieprzemienną # nieabelową
Goyahkla # Goyathlay
prosień # prosiniec
role-playing game # grą wyobraźni
Gęstość optyczna # absorbancją
Grawitacja # ciążeniem powszechnym
Graduał # Graduale
GG # gg
He # hej
Haller # Heller
Hb # HGB
żyjące kwiaty # kadō
sty

### Synonimy

In [447]:
synonims = [
    "Alternatywa # suma logiczna",
    "pytania # sugestie",
    "Mediana # wartością środkową",
    "kontrapozycja # transpozycja",
    "odbitka # kalka",
    "Skraplanie # kondensacja",    
    "iloczyn # przecięcie",
    "Ubezpłodnienie # sterylizacja",
    "Urodzajny # Owocny",
    "porażenie słoneczne # udar cieplny",
    "posłaniec # anioł",
    "łazienka # toaleta"
    "Skutek uboczny # efekt uboczny",
    "translatoryka # przekładoznawstwo",
    "silnik wysokoprężny # silnik Diesla",
    "Trójkątny kapelusz # Trójgraniasty kapelusz",
    "synteza jądrowa # fuzja jądrowa",   
    "święta fajka # fajka pokoju",
    "Goniec # laufrem",
    "e-terapia # cyberterapia",
    "Otwarcie szachowe # debiutem",
    "Kapusta warzywna # kapusta ogrodowa",
    "Precesja # ruch precesyjny"
    "Element nilpotentny # nilpotent",
    "Szachownicowym zabójcą # Bitcewskim maniakiem",
    "Kószka # bezdenka",
    "surowy # niefarbowany",
    "Śluzowiak # torbiel śluzowa",
    "Rękawica # Pięciopalcówką",
    "Sialoza # sialoadenoza",
    "skupienie # ogniskowanie",
    "Szablon funkcji # funkcja szablonowa"
    
]
len(synonims)

30

### Niesynonimy

In [461]:
non_synonims = [
    "podnieść # opuścić",
    "wrześniowy # czerwony",
    "model # modelka",
    "tak # nie",
    "Reprezentacja Wenezueli # La Vinotinto",
    "Rajd św. Mikołaja # efekt grudnia",
    "horror # fantasy",
    "diabeł # duch",
    "do # portal",
    "Ta ta # Ta tam",
    "kakaowym # czekoladowym",
    "CSV # PCS",
    "Jedediah Strong # Jed Smith",
    "Ryszard Szawłowski # Richard Szawlowski",
    "i # u",
    "Dłubanie w nosie # rhinotillexis",
    "zombie shuffle # zombie pub crawl",
    "Open Space Technology # Open Space",
    "T-600 # terminator z serii 600",
    "Step Up 2 # 27 sukienek",
]
len(non_synonims)

20

## Task 2

In [263]:
with open("znaki_wikipedii.txt") as fstream:
    char_lists = list(map(lambda x: x.strip().split(' '), fstream.readlines()))
    chars = list(itertools.chain(*char_lists))


def group_characters(chars):
    groups = defaultdict(lambda: set())
    for c in chars:
        try:
            char_category = unicodedata.category(c[0])
            char_name = unicodedata.name(c[0])
            if char_category == "So":
                groups["symbol"].add(c)
            elif char_category == "Sm":
                groups["math"].add(c)        
            elif char_category in ["Pc", "Pd", "Pe", "Pf", "Pi", "Po", "Ps"]:
                groups["punctuation"].add(c)        
            elif char_category == "Sc":
                groups["currency"].add(c)
            elif char_category == "Sk":
                groups["modifier"].add(c)                
            elif char_category in ["Nd", "NI", "No"]:
                groups["number"].add(c)           
            elif char_category == "Cf":
                groups["format"].add(c)        
            elif char_category in ["Zs", "Zl"]:
                groups["separator"].add(c)            
            else:
                groups[char_name.split(' ')[0]].add(c)
        except:
            pass
    return groups


groups = group_characters(chars)
print("Number of groups: ", len(groups))
for key, val in groups.items():
    print(f"Name: {key}, characters: {val}\n")

Number of groups:  83
Name: math, characters: {'⁼', '∇', '≧', '−', '⊛', '←', '⊂', '∋', '∀', '⋰', '⩽', '⁻', '＞', '∞', '∽', '≅', '℘', '⨁', '⌡', '⊥', '⊗', '≋', '∴', '∫', '＝', '⩾', '≉', '∨', '≈', '＜', '∈', '≥', '⊙', '⊐', '∛', '∓', '∙', '∂', '≡', '⁺', '◁', '⊏', '∵', '⧸', '≪', '∏', '√', '⊤', '∿', '¬', '+', '⊞', '>', '⊣', '≤', '⊢', '∝', '≦', '⎯', '⋐', '∜', '⌠', '⧹', '↓', '₊', '≣', '⋆', '×', '▷', '|', '∗', '⋱', '÷', '≐', '↑', '⋑', '~', '≠', '＋', '⊘', '⫺', '∅', '∑', '∎', '<', '｜', '₋', '⋯', '⊽', '⨧', '⇒', '=', '⫹', '⊄', '⇔', '∆', '∪', '₌', '≫', '∘', '∃', '⋮', '～', '⋅', '∉', '↔', '↦', '⊆', '→', '⊃', '♯', '±', '≝', '∠', '≛', '⁄', '≃', '∟', '⊇', '∩', '∧', '﹢', '∼', '⊿', '϶', '⊕', '⁒', '∕', '⊳'}

Name: LATIN, characters: {'k', 'R', 'Ớ', 'Ď', 'ẋ', 'Ṱ', 'ắ', 'ꝼ', 'ᵶ', 'ť', 'ꞑ', 'Ǜ', 'ṁ', 'ḱ', 'ꝗ', 'Ǒ', 'ḵ', 'Ü', 'Ê', 'Ḫ', 'Ƨ', 'ᶊ', 'Ḷ', 'Ÿ', 'ǿ', 'ʊ', 'ɟ', 'Ƈ', 'ʩ', 'Ỗ', 'Ɀ', 'Ơ', 'È', 'Ľ', 'Ứ', 'I', 'ś', 'İ', 'Ĝ', 'ₔ', 'Ẹ', 'J', 'Ķ', 'ḁ', 'ỡ', 'Ȕ', 'Ƙ', 'Ƥ', 'Ự', 'Ḩ', 'Ś', 'ŗ', 'ỹ', 'ƙ', 'Ȑ', 'ç', '

## Task 3

In [29]:
with open("cytaty.txt") as fstream:
    citations = list(map(lambda x: x.strip(), fstream.readlines()))
len(citations)

197570

In [33]:
citations[10], citations[15], citations[20]

('* 10-go maja 1943 roku kończy się pierwszy okres krwawej historii Żydów warszawskich, pierwszy okres naszej krwawej historii. Miejsce, gdzie niegdyś było getto, staje się równą górą gruzów, sięgających drugiego piętra.&lt;br /&gt;Ci, którzy polegli, spełnili swoje zadanie do końca, do ostatniej kropli krwi, która wsiąkła w bruk getta warszawskiego.&lt;br /&gt;My, którzy ocaleli, Wam pozostawiamy to, by pamięć o nich nie zginęła.',
 '* 10. Przyjmując Prawdę (Satya), realizujesz Ścieżkę Mistrza (Guru Margha), osiągając Oświecenie (Tattva) dla wszystkich czujących istot w całym wszechświecie.',
 '* 11. Będąc w najwyższym, najgłębszym stanie umysłu (Czitta), dostrzegasz różnorodne nauki i wyzwalasz się ze wszystkich ograniczeń.')

In [91]:
def get_without_punctuation(word):
    start_index, end_index = len(word), 0
    for i in range(len(word)):
        if word[i] not in string.punctuation:
            start_index = i
            break
    for i in range(len(word) - 1, 0, -1):
        if word[i] not in string.punctuation:
            end_index = i
            break
    return word[start_index:(end_index + 1)]


def space_punct_tokenize(text):
    return list(filter(len, map(get_without_punctuation, text.split())))


def compare_citations(citation1, citation2):
    set_of_citation1 = set(citation1)
    set_of_citation2 = set(citation2)    
    only_citation1, only_citation2 = [], []
    for word in citation1:
        if word not in set_of_citation2:
            only_citation1.append(word)
    for word in citation2:
        if word not in set_of_citation1:
            only_citation2.append(word)
    print(f"Only in citation1: {only_citation1}\n")
    print(f"Only in citation2: {only_citation2}\n\n")

In [ ]:
nltk_citations = list(map(nltk.tokenize.word_tokenize, citations))

In [73]:
space_punct_citations = list(map(space_punct_tokenize, citations))

In [94]:
print(f"{citations[10]}\n")
compare_citations(nltk_citations[10], space_punct_citations[10])

* 10-go maja 1943 roku kończy się pierwszy okres krwawej historii Żydów warszawskich, pierwszy okres naszej krwawej historii. Miejsce, gdzie niegdyś było getto, staje się równą górą gruzów, sięgających drugiego piętra.&lt;br /&gt;Ci, którzy polegli, spełnili swoje zadanie do końca, do ostatniej kropli krwi, która wsiąkła w bruk getta warszawskiego.&lt;br /&gt;My, którzy ocaleli, Wam pozostawiamy to, by pamięć o nich nie zginęła.

Only in citation1: [',', '.', ',', ',', ',', 'piętra.', '&', 'lt', ';', 'br', '/', '&', 'gt', ';', 'Ci', ',', ',', ',', ',', 'warszawskiego.', '&', 'lt', ';', 'br', '/', '&', 'gt', ';', 'My', ',', ',', ',', '.']

Only in citation2: ['piętra.&lt;br', 'gt;Ci', 'warszawskiego.&lt;br', 'gt;My']




In [95]:
print(f"{citations[15]}\n")
compare_citations(nltk_citations[15], space_punct_citations[15])

* 10. Przyjmując Prawdę (Satya), realizujesz Ścieżkę Mistrza (Guru Margha), osiągając Oświecenie (Tattva) dla wszystkich czujących istot w całym wszechświecie.

Only in citation1: ['.', '(', ')', ',', '(', ')', ',', '(', ')', '.']

Only in citation2: []




In [466]:
print(f"{citations[10]}\n")
compare_citations(nltk_citations[20], space_punct_citations[20])
print(nltk_citations[20])
print(space_punct_citations[20])

* 10-go maja 1943 roku kończy się pierwszy okres krwawej historii Żydów warszawskich, pierwszy okres naszej krwawej historii. Miejsce, gdzie niegdyś było getto, staje się równą górą gruzów, sięgających drugiego piętra.&lt;br /&gt;Ci, którzy polegli, spełnili swoje zadanie do końca, do ostatniej kropli krwi, która wsiąkła w bruk getta warszawskiego.&lt;br /&gt;My, którzy ocaleli, Wam pozostawiamy to, by pamięć o nich nie zginęła.

Only in citation1: ['.', ',', '(', ')', ',', '.']

Only in citation2: []


['*', '11', '.', 'Będąc', 'w', 'najwyższym', ',', 'najgłębszym', 'stanie', 'umysłu', '(', 'Czitta', ')', ',', 'dostrzegasz', 'różnorodne', 'nauki', 'i', 'wyzwalasz', 'się', 'ze', 'wszystkich', 'ograniczeń', '.']
['*', '11', 'Będąc', 'w', 'najwyższym', 'najgłębszym', 'stanie', 'umysłu', 'Czitta', 'dostrzegasz', 'różnorodne', 'nauki', 'i', 'wyzwalasz', 'się', 'ze', 'wszystkich', 'ograniczeń']


In [465]:
print(f"{citations[200]}\n")
compare_citations(nltk_citations[100], space_punct_citations[100])

* 21 marca 2002 r. zniknęła bez śladu 13-letnia uczennica Milly Dowler.(...) Policja znalazła gwałciciela i mordercę dopiero w 2008 r., ale śledztwo szło tak powoli, że skazano go dopiero w 2011 r. Znów wyglądało na to, że policja zawiodła i że nie było to dziełem przypadku. Okazało się, że dziennikarze „News of the World” cały czas odsłuchiwali pocztę głosową dziewczynki. To, że ktoś słucha tych wiadomości i kasuje poprzednie, dawało rodzinie fałszywą nadzieję, że ich córka jeszcze żyje. Wciąż nagrywali dla niej kolejne wiadomości, z których brukowce ukręcały wariackie teorie.

Only in citation1: ['``', "'1966", ':', "'", "''"]

Only in citation2: ['1966']


